In [1]:
import pandas as pd
#import numpy as np
import os
#import sys
#import matplotlib.pyplot as plt
#import matplotlib
from selenium import webdriver
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
import random
import time
from datetime import datetime

#### 0. 환경설정

In [16]:
delay = 1.5   # 딜레이 시간

#### 1. 초기화

In [2]:
driver = webdriver.Chrome('chromedriver')

In [3]:
# result 폴더 생성
if not any([s == 'result' for s in os.listdir('.')]):
    os.mkdir('result')
    
# source 폴더 생성
if not any([s == 'source' for s in os.listdir('.')]):
    os.mkdir('source')

In [15]:
# 접속하기
url = 'http://www.encar.com/ev/ev_carsearchlist.do?carType=ev&searchType=model&TG.R=D#!%7B%22action%22%3A%22(And.Hidden.N._.CarType.A._.FuelType.%EC%A0%84%EA%B8%B0._.(C.GreenType.Y._.EvType.%EC%A0%84%EA%B8%B0%EC%B0%A8.))%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22ModifiedDate%22%2C%22page%22%3A1%2C%22limit%22%3A%2250%22%7D'
driver.get(url)

# [20개씩 보기] → [50개씩 보기]로 변환
viewer = Select(driver.find_element_by_css_selector('select#pagerow'))
viewer.select_by_value('50')
time.sleep(delay)

# 데이터 url 수집
batch = []
page = 1
while(True):
    try:
        driver.find_element_by_css_selector('div#pagination').find_element_by_xpath('//a[@data-page="{}"]'.format(page)).click()
    except:
        break
    time.sleep(delay)
    
    rows = driver.find_element_by_css_selector('tbody#sr_normal').find_elements_by_tag_name('tr')
    for row in rows:
        car_url = row.find_element_by_css_selector('td.inf > a').get_attribute('href')
        batch.append(car_url)
    page += 1

# url 저장
now = datetime.now().strftime('%Y%m%d%H%M%S')

# 결과 저장하기
df = pd.DataFrame(batch)
writer = pd.ExcelWriter('./result/car_url_{}.xlsx'.format(now), 'xlsxwriter')
df.to_excel(writer, index=False)
writer.save()
writer.close()

#### 2. page 저장하기

In [4]:
# URL 데이터 불러오기
df = pd.read_excel('./result/car_url_20190903162242.xlsx')

# 데이터 저장하기
for i, url in df.iterrows():
    driver.get(url[0])
    time.sleep(2 + 3*random.random())
    with open('./source/car_{}.html'.format(i+1), 'w', -1, encoding='utf-8') as f:
        f.write(driver.page_source)

#### 3. page에서 데이터 추출하기

In [7]:
from bs4 import BeautifulSoup

In [5]:
with open('./source/car_1.html', 'r', encoding='utf-8') as f:
    html = f.read()

In [145]:
soup = BeautifulSoup(html[html.find('<div id="areaBase"'):html.find('<div class="product_detail"')])
result = {}
result['브랜드'] = soup.select('div.product_left > div.area_info > h1.prod_name > span.brand')[0].text
result['디테일'] = soup.select('div.product_left > div.area_info > h1.prod_name > span.detail')[0].text
prod_infomain_list = soup.select('div.product_left > div.area_info > div.prod_infomain > ul > li')
prod_infomain = {}
for p in prod_infomain_list:
    key = p.select('span.blind')[0].text.replace(':', '')
    value = p.text.replace(' ', '').replace('\n', '').replace('자세히보기', '').replace(p.select('span.blind')[0].text, '')
    prod_infomain[key] = value
result.update(prod_infomain)
result['가격'] = soup.select('div.product_left > div.area_info > div.prod_price > span.num')[0].text
result['등록번호'] = soup.select('div.product_left > div.area_info2nd > div.prod_infoetc > ul.reg > li')[0].text.replace('등록번호', '').replace('\n', '').replace('자세히보기', '').replace(' ', '')
result['조회수'] = soup.select('div.product_left > div.area_info2nd > div.prod_infoetc > ul.reg > li')[1].text.replace('조회수', '').replace('\n', '').replace('자세히보기', '').replace(' ', '')
result['찜'] = soup.select('div.product_left > div.area_info2nd > div.prod_infoetc > ul.reg > li')[2].text.replace('찜', '').replace('\n', '').replace('자세히보기', '').replace(' ', '')
result['핫마크'] = list(map(lambda x: x.text, soup.select('div.product_left > div.area_info2nd > div.prod_addon > span')))
result

{'브랜드': '기아 쏘울 EV',
 '디테일': '  EV ',
 '주행거리': '29,000Km',
 '연식': '16년11월식(17년형)',
 '연료': '전기',
 '차종': '준중형차',
 '배기량': '75cc',
 '변속기': '오토',
 '색상': '흰색투톤',
 '차량번호': '48저7664',
 '가격': '1,630만원',
 '등록번호': '25345659',
 '조회수': '101',
 '찜': '0',
 '핫마크': ['hot09', 'hot26', 'hot29']}

In [206]:
soup = BeautifulSoup(html[html.find('<div class="product_detail"'):html.find('<div><div id="ad_btm">')])
option_classes = soup.select('div#areaOption > div.box_opt > div.con.option_hover')
option_effective = []
for option_class in option_classes:
    options = option_class.select('dl > dd.on')
    for option in options:
        try:
        #option_effective.append()
            print(option.select('a')[0].text.replace('\n', '').replace(' ', ''))
        except:
            print(option.select('p')[0].text.replace('\n', '').replace(' ', ''))

전동접이사이드미러
알루미늄휠
열선스티어링휠
스티어링휠리모컨
ECM룸미러
하이패스
파워도어록
파워스티어링휠
파워윈도우
에어백(운전석,동승석)
에어백(사이드)
에어백(커튼)
브레이크잠김방지(ABS)
미끄럼방지(TCS)
차체자세제어장치(ESC)
타이어공기압센서(TPMS)
주차감지센서(전방,후방)
후방카메라
크루즈컨트롤(일반)
전자식주차브레이크(EPB)
자동에어컨
스마트키
무선도어잠금장치
레인센서
오토라이트
내비게이션
앞좌석AV모니터
블루투스
USB단자
AUX단자
가죽시트
열선시트(앞좌석,뒷좌석)
통풍시트(운전석,동승석)
버튼시동키,
